In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_diagonal_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,636883.32,-1.20,0.52,-1.89,0.29,0.18,0.08,-0.04,-0.01
3,-1.20,99886.08,4.74,-0.06,0.40,-0.03,0.02,0.01,0.00
4,0.52,4.74,21895.32,0.07,0.12,-0.03,0.01,-0.00,0.00
5,-1.89,-0.06,0.07,2368.22,0.05,0.03,-0.01,-0.00,-0.00
6,0.29,0.40,0.12,0.05,634.68,0.01,-0.00,0.00,-0.00
7,0.18,-0.03,-0.03,0.03,0.01,178.90,0.00,-0.00,-0.00
8,0.08,0.02,0.01,-0.01,-0.00,0.00,40.76,-0.00,-0.00
9,-0.04,0.01,-0.00,-0.00,0.00,-0.00,-0.00,5.64,-0.00
10,-0.01,0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.34


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00000,0.00000,-0.00005,0.00001,0.00002,0.00002,-0.00002,-0.00001
3,-0.00000,1.00000,0.00010,-0.00000,0.00005,-0.00001,0.00001,0.00001,0.00002
4,0.00000,0.00010,1.00000,0.00001,0.00003,-0.00002,0.00001,-0.00001,0.00001
5,-0.00005,-0.00000,0.00001,1.00000,0.00004,0.00005,-0.00004,-0.00004,-0.00002
6,0.00001,0.00005,0.00003,0.00004,1.00000,0.00002,-0.00003,0.00001,-0.00001
7,0.00002,-0.00001,-0.00002,0.00005,0.00002,1.00000,0.00002,-0.00001,-0.00002
8,0.00002,0.00001,0.00001,-0.00004,-0.00003,0.00002,1.00000,-0.00003,-0.00000
9,-0.00002,0.00001,-0.00001,-0.00004,0.00001,-0.00001,-0.00003,1.00000,-0.00004
10,-0.00001,0.00002,0.00001,-0.00002,-0.00001,-0.00002,-0.00000,-0.00004,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

7.491859070088174

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.50875601e+10 1.09195164e+09 5.13299268e+07 5.85950934e+05
 4.03889877e+04 3.00262868e+03 1.38243882e+02 1.98776073e+00
 1.49091703e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,9.997889e-01,-0.020086,-0.004286,-0.000466,-0.000124,-0.000035,-0.000008,-0.000001,-7.789138e-08
1,1.993295e-02,0.999255,-0.032876,-0.003370,-0.000899,-0.000255,-0.000058,-0.000008,-4.623169e-07
2,4.932090e-03,0.032711,0.999271,-0.018348,-0.004850,-0.001369,-0.000311,-0.000043,-2.569166e-06
3,6.123419e-04,0.003887,0.017895,0.998063,-0.057455,-0.014898,-0.003380,-0.000469,-2.869208e-05
4,1.960184e-04,0.001247,0.005707,0.055989,0.995327,-0.076810,-0.015860,-0.002182,-1.336345e-04
5,6.940207e-05,0.000444,0.002027,0.018794,0.074305,0.993718,-0.080847,-0.010435,-6.371810e-04
6,2.112417e-05,0.000135,0.000618,0.005725,0.021410,0.078419,0.994119,-0.071188,-4.222435e-03
7,4.462915e-06,0.000028,0.000129,0.001190,0.004431,0.015730,0.069686,0.995540,-6.144246e-02
8,5.522038e-07,0.000003,0.000016,0.000146,0.000542,0.001924,0.008442,0.060977,9.981015e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00021106, 0.00074546, 0.00072851, 0.0019371 , 0.00467346,
       0.0062819 , 0.00588063, 0.00445985, 0.00189852])